## DataLoader Class
The DataLoader Class loads data into a SQLite database from files. All files in the folder should be in the same format and they should be csv files or zipped csv files or any other file type in csv format

In [1]:
# Import libraries
import pandas as pd
import os
from sqlalchemy import create_engine
from loaders import DataLoader

## Parameters that could be changed

In [2]:
DIRECTORY = 'data' # Folder in which files are saved
DBNAME = 'data.sqlite3' # Name of the database
TABLENAME = 'eod' # Name of the table
UNIVERSE = 'NIFTY50'
STOP_LOSS = 4
NUM_STOCKS = 5
CAPITAL = 20000
LEVERAGE = 1
REF_PRICE = 'close'
COMMISSION = 0 # In percent

Don't change the below cell; Just run it.

This updates data of all the files in the folder. If you add a file, just rerun it again so that the database is updated.

In [3]:
connection_string = 'sqlite:///' + DBNAME
engine = create_engine(connection_string)
dl = DataLoader(DIRECTORY, mode='SQL', engine=engine, tablename=TABLENAME)
dl.load_data(parse_dates=['TIMESTAMP'], usecols=range(13)) # This actually loads data

In [4]:
# Now read the entire table back
df = pd.read_sql_table(TABLENAME, engine)

# Check the number of rows; would increase as you put more files in your folder
len(df)

43802

Don't change any cells below.
Just change the **PARAMETERS** above

Of course, you could always *hack the below code*

Backtesting
------------
1. Filter data only for our stock universe
2. Calculate daily returns for each of the stock
3. Carry the returns figure to the next day so that we can view them the next day
4. Group by each day, and apply our strategy
5. Append all the data
6. Calculate prices and performance
7. Evaluate metrics

In [5]:
symbols = pd.read_excel('universe.xlsx', sheet_name=UNIVERSE, header=None).values.ravel()

df = df[df['symbol'].isin(symbols)]
df = df[df['series'] == "EQ"].reset_index(drop=True)
df['ret'] = (df['close']/df['prevclose']) - 1
df = df.sort_values(by='timestamp')
df['ret'] = df.groupby('symbol')['ret'].transform(lambda x: x.shift(1))
grouped = df.groupby('timestamp')
collect = []
for name, group in grouped:
    temp = group.sort_values(by='ret', ascending=False).iloc[:5]
    collect.append(temp)
orders = pd.concat(collect)
    

In [6]:
def isPrice(price, high, low):
    if price >= low and price <=high:
        return True
    else:
        False

In [7]:
trading_capital = CAPITAL * LEVERAGE
orders['price'] = orders['open']
orders['stop_loss'] = (orders['price'] * (1+STOP_LOSS*0.01)).round(2)
orders['qty'] = (trading_capital/NUM_STOCKS/orders['price']).round()
orders['sell'] = [price if isPrice(price, high, low) else close for
                 price, high, low, close in 
                 zip(orders['price'], orders['high'], orders['low'], orders[REF_PRICE])]
orders['buy'] = [price if isPrice(price, high, low) else close for
                 price, high, low, close in 
                 zip(orders['stop_loss'], orders['high'], orders['low'], orders[REF_PRICE])]
orders['profit_per_unit'] = orders['sell'] - orders['buy']
orders['total_profit'] = orders['profit_per_unit'] * orders['qty']
cols = ['timestamp', 'symbol', 'buy', 'sell', 'profit_per_unit', 'total_profit']


## Metrics

In [9]:
orders.tail()[['symbol', 'open', 'high', 'low', 'close', 
               'qty', 'buy', 'sell', 'total_profit']]

,symbol,open,high,low,close,qty,buy,sell,total_profit
1060,DRREDDY,2510.0,2632.0,2495.00,2593.40,2.0,2610.40,2510.0,-200.80
1092,TECHM,768.0,776.5,741.50,745.65,5.0,745.65,768.0,111.75
1080,LUPIN,936.5,957.9,931.55,936.10,4.0,936.10,936.5,1.60
1089,TATAMOTORS,271.4,273.0,265.25,266.60,15.0,266.60,271.4,72.00
1064,HCLTECH,1053.0,1061.0,1044.60,1049.15,4.0,1049.15,1053.0,15.40


In [10]:
total_profit = orders['total_profit'].sum()
'Returns for the period = {:.2f}%'.format((total_profit/CAPITAL)*100)

'Returns for the period = 4.11%'

In [15]:
orders[orders['open'] == orders['high']].total_profit.sum()

613.7999999999997

In [12]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
by_day = orders.groupby('timestamp').total_profit.sum()
(by_day.cumsum() + CAPITAL).plot(title = 'Portfolio by day')
plt.xlabel('Date')
plt.ylabel('Capital')
plt.show()

<Figure size 640x480 with 1 Axes>

In [13]:
orders.corr()

,open,high,low,close,last,prevclose,tottrdqty,tottrdval,totaltrades,ret,price,stop_loss,qty,sell,buy,profit_per_unit,total_profit
open,1.000000,0.999981,0.999974,0.999957,0.999960,0.999999,-0.148897,-0.069653,-0.150378,0.174624,1.000000,1.000000,-0.351303,1.000000,0.999955,0.892213,-0.047230
high,0.999981,1.000000,0.999985,0.999990,0.999991,0.999978,-0.149135,-0.068445,-0.149667,0.174166,0.999981,0.999981,-0.353789,0.999981,0.999989,0.889674,-0.050773
low,0.999974,0.999985,1.000000,0.999990,0.999990,0.999973,-0.150328,-0.069683,-0.150962,0.173019,0.999974,0.999974,-0.354394,0.999974,0.999989,0.889338,-0.049820
close,0.999957,0.999990,0.999990,1.000000,1.000000,0.999954,-0.150318,-0.068979,-0.150405,0.173123,0.999957,0.999957,-0.355375,0.999957,1.000000,0.888022,-0.052584
last,0.999960,0.999991,0.999990,1.000000,1.000000,0.999956,-0.150334,-0.069011,-0.150401,0.173261,0.999960,0.999960,-0.355334,0.999960,0.999999,0.888153,-0.052516
prevclose,0.999999,0.999978,0.999973,0.999954,0.999956,1.000000,-0.148804,-0.069727,-0.150461,0.174287,0.999999,0.999999,-0.350977,0.999999,0.999952,0.892328,-0.047054
tottrdqty,-0.148897,-0.149135,-0.150328,-0.150318,-0.150334,-0.148804,1.000000,0.887464,0.872605,0.081900,-0.148897,-0.148897,0.246630,-0.148897,-0.150382,-0.061490,0.049446
tottrdval,-0.069653,-0.068445,-0.069683,-0.068979,-0.069011,-0.069727,0.887464,1.000000,0.950677,0.125807,-0.069653,-0.069653,-0.081951,-0.069653,-0.069022,-0.092186,-0.118504
totaltrades,-0.150378,-0.149667,-0.150962,-0.150405,-0.150401,-0.150461,0.872605,0.950677,1.000000,0.172677,-0.150378,-0.150378,0.019020,-0.150378,-0.150473,-0.129302,-0.126903
ret,0.174624,0.174166,0.173019,0.173123,0.173261,0.174287,0.081900,0.125807,0.172677,1.000000,0.174624,0.174624,-0.017758,0.174624,0.173257,0.221006,0.051175
